# IMDB DataSet Retriever

## Download IMDB Data and load into Pandas

In [1]:
!curl -o title.basics.tsv.gz https://datasets.imdbws.com/title.basics.tsv.gz
!curl -o title.ratings.tsv.gz https://datasets.imdbws.com/title.ratings.tsv.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  187M  100  187M    0     0  65.6M      0  0:00:02  0:00:02 --:--:-- 65.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7320k  100 7320k    0     0  51.4M      0 --:--:-- --:--:-- --:--:-- 51.8M


In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import gzip, shutil
import pandas as pd

with gzip.open("title.basics.tsv.gz", "rb") as f_in:
    with open("title.basics.tsv", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

with gzip.open("title.ratings.tsv.gz", "rb") as f_in:
    with open("title.ratings.tsv", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

basics = pd.read_csv("title.basics.tsv", sep="\t", low_memory=False, na_values=["\\N"])
ratings = pd.read_csv(
    "title.ratings.tsv", sep="\t", low_memory=False, na_values=["\\N"]
)
full_data = pd.merge(basics, ratings, on="tconst")
samples = full_data.sample(n=100, random_state=42)
samples.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
1233913,tt4902180,tvEpisode,The Birth of the Free Market,The Birth of the Free Market,0.0,2014.0,NaN,320,"Documentary,History",6.9,9
189343,tt0312012,video,We're Coming to Help,We're Coming to Help,0.0,2002.0,NaN,51,"Drama,Sci-Fi",6.0,49
1473704,tt9420822,video,WSU 2nd Anniversary Show,WSU 2nd Anniversary Show,0.0,2009.0,NaN,123,"Game-Show,Sport,Talk-Show",9.0,10
119891,tt0173675,tvMiniSeries,Bitva za Moskvu,Bitva za Moskvu,0.0,1985.0,1985.0,358,"Drama,History,War",7.3,486
1103324,tt3188128,tvEpisode,Extreme Howdy's Makeover,Extreme Howdy's Makeover,0.0,2013.0,NaN,21,"Comedy,Family",7.5,229


In [5]:
print(samples.shape[0])

100


## Load data from Pandas to SQLLite

In [6]:
import pandas as pd
import sqlite3

# SQLite 데이터베이스 연결 및 커서 생성
conn = sqlite3.connect("example.db")
cursor = conn.cursor()

# 데이터프레임을 SQLite 테이블로 저장
samples.to_sql("my_table", conn, index=False, if_exists="replace")

# 테이블에 저장된 데이터 출력
query = "SELECT * FROM my_table"
result = pd.read_sql_query(query, conn)
print(result)

# 연결 종료
conn.close()

       tconst     titleType                           primaryTitle  \
0   tt4902180     tvEpisode           The Birth of the Free Market   
1   tt0312012         video                   We're Coming to Help   
2   tt9420822         video               WSU 2nd Anniversary Show   
3   tt0173675  tvMiniSeries                        Bitva za Moskvu   
4   tt3188128     tvEpisode               Extreme Howdy's Makeover   
..        ...           ...                                    ...   
95  tt0036034         movie                    The Living Magoroku   
96  tt1647733     tvEpisode                Little Miss Nightingale   
97  tt1190404     tvEpisode                          The Agreement   
98  tt0402991         short                     The Empty Building   
99  tt1251591     tvEpisode  No Beads, No Babes, No Bourbon Street   

                            originalTitle  isAdult  startYear  endYear  \
0            The Birth of the Free Market      0.0     2014.0      NaN   
1          

## Generate the SQL query by using LLM

In [9]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.sql_database.query import create_sql_query_chain   

OPEN_AI_APIKEY = "{YOUR_OPENAI_KEY}"
model = ChatOpenAI(api_key=OPEN_AI_APIKEY)

db = SQLDatabase.from_uri("sqlite:///example.db")
chain = create_sql_query_chain(model, db, k=20) | StrOutputParser()
result = chain.invoke(
    {
        "question": """Please provide a list of  movies that have an averageRating of 8.0 or higher 
                         and have been commercially available since 2008."""
    }
)
print(result)

SQLQuery: SELECT "tconst", "primaryTitle", "averageRating", "startYear" FROM my_table WHERE "averageRating" >= 8.0 AND "startYear" >= 2008 LIMIT 20;
